□ [nn-morse](https://github.com/pd0wm/nn-morse)で学習したモデル(onnx形式)からonnxruntimeだけで推論してみる

■ GoogleDriveに接続

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


■ 環境構築

In [2]:
!cp drive/MyDrive/MORSE/新models-8/main8.py main.py
!cp drive/MyDrive/MORSE/新models-8/my_morse.py morse.py
!cp "drive/MyDrive/MORSE/新models-8/nn-morse.onnx" nn-morse.onnx

!cp drive/MyDrive/MORSE/hsu-sat1.wav .

■ onnxruntimeから推論 part1
1. onnx形式のモデルを読み込む

In [3]:
!pip install onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.9 MB/s eta 0:00:00


3. WAVファイルの読み込み

In [5]:
import numpy as np
import scipy.io.wavfile
from scipy import signal

#from main import Net, num_tags, prediction_to_str
from main import num_tags, prediction_to_str
from morse import ALPHABET, SAMPLE_FREQ, get_spectrogram

rate, data = scipy.io.wavfile.read("hsu-sat1.wav")

# Resample and rescale
length = len(data) / rate
new_length = int(length * SAMPLE_FREQ)

data = signal.resample(data, new_length)
data = data.astype(np.float32)
data /= np.max(np.abs(data))

# Create spectrogram
spec = get_spectrogram(data)
spec_orig = spec.copy()
spectrogram_size = spec.shape[0]

- numpyでspecを加工するよう修正

In [6]:
spec = spec.transpose(1,0)
print (spec)
spec = np.expand_dims(spec, axis=0)
print (spec)

[[9.3473445e-06 3.5811576e-07 2.5687217e-07 ... 2.0474295e-06
  2.3952559e-06 4.5287343e-05]
 [2.7729541e-11 9.7814682e-13 6.5935278e-13 ... 5.9861293e-07
  7.6512720e-07 2.0086612e-05]
 [5.8276379e-12 8.8115744e-11 6.9371169e-11 ... 4.4178103e-07
  5.5890860e-07 1.4939497e-05]
 ...
 [8.4093458e-04 3.7331714e-03 3.4323842e-03 ... 4.6604257e-03
  4.9493425e-03 5.4418179e-04]
 [2.9688408e-05 1.9083964e-03 6.4633577e-03 ... 2.6144108e-04
  7.1848733e-03 6.0275874e-05]
 [4.1993064e-04 4.9391231e-03 8.4552309e-03 ... 6.6844317e-05
  1.1158085e-03 2.6162663e-03]]
[[[9.3473445e-06 3.5811576e-07 2.5687217e-07 ... 2.0474295e-06
   2.3952559e-06 4.5287343e-05]
  [2.7729541e-11 9.7814682e-13 6.5935278e-13 ... 5.9861293e-07
   7.6512720e-07 2.0086612e-05]
  [5.8276379e-12 8.8115744e-11 6.9371169e-11 ... 4.4178103e-07
   5.5890860e-07 1.4939497e-05]
  ...
  [8.4093458e-04 3.7331714e-03 3.4323842e-03 ... 4.6604257e-03
   4.9493425e-03 5.4418179e-04]
  [2.9688408e-05 1.9083964e-03 6.4633577e-03 ... 2

4. 推論

In [7]:
import onnxruntime as ort

sess = ort.InferenceSession('nn-morse.onnx')
output = sess.run(['output'], {'input': spec})

In [10]:
print ("◇output")
print (output)

output1 = np.array(output[0][0])
print ("◆y_pred⇐output1")
print (output1)

y_pred_l = np.exp(output1.tolist())
print ("◆y_pred_l")
print (y_pred_l)


◇output
[array([[[ 0.0000000e+00, -2.1061573e+01, -2.0310665e+01, ...,
         -2.4417107e+01, -2.4160883e+01, -2.4291817e+01],
        [ 0.0000000e+00, -3.8936195e+01, -3.3078651e+01, ...,
         -4.3582199e+01, -4.2318993e+01, -3.7926727e+01],
        [ 0.0000000e+00, -3.8175545e+01, -3.1148098e+01, ...,
         -3.9882446e+01, -3.8202049e+01, -3.4655579e+01],
        ...,
        [-1.0794534e-03, -1.0878096e+01, -1.1868061e+01, ...,
         -1.2584034e+01, -1.1283299e+01, -1.3148109e+01],
        [-9.0045907e-04, -1.2186127e+01, -1.1783067e+01, ...,
         -1.2425255e+01, -1.2001100e+01, -1.2931832e+01],
        [-6.7107310e-03, -9.6853209e+00, -9.5085764e+00, ...,
         -1.0000663e+01, -9.1748743e+00, -1.0216950e+01]]], dtype=float32)]
◆y_pred⇐output1
[[ 0.0000000e+00 -2.1061573e+01 -2.0310665e+01 ... -2.4417107e+01
  -2.4160883e+01 -2.4291817e+01]
 [ 0.0000000e+00 -3.8936195e+01 -3.3078651e+01 ... -4.3582199e+01
  -4.2318993e+01 -3.7926727e+01]
 [ 0.0000000e+00 -3.817554

5. 文字列に変換

In [11]:
# Convert prediction into string
# TODO: proper beam search
m = np.argmax(output1, 1)
print ("◆m")
print (m)

print ("変換")
print(prediction_to_str(m))

◆m
[0 0 0 ... 0 0 0]
変換
H4.18V0.08A8.14DEEEEETTTTEE  0JS1YHSHSUSAT104.19V0.10A8.46DEEEEETETTTE 9I   U  J4 V1 O: EDI       ISW               1:   I   


6. pytorchの結果と比較.同じ結果になった！

```
E8V.08A8.14D   ETTE                     OJS1YHSHSUSAT104.19V0.10A8.46DEEEEETETTTE                                             IA            M  
```